<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/P1-NL2SQL/nl2sql_prompt_SQLCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">Learn by Doing LLM Projects</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Natural Language to SQL with OpenSource LLMs</h2>
    <h3>Create a prompt to generate SQL with defog SQLCoder 7B Model</h3>
    by <b>Pere Martra</b>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

Starting with a [prompt specially designed for OpenAI](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/P1-NL2SQL/nl2sql_enhaced_ohaio.ipynb), we adapt it for a model like SQLCoder 7B.

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [1]:
# Install the necesary libraries:
!pip -q install accelerate bitsandbytes
!pip -q install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 905.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00


In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

This function wraps the call to *model.generate*

In [9]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

#Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [10]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [11]:
sp_nl2sql = sp_nl2sql.format(question="Return The name of the best paid employee")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee 

In [12]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [18]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [20]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [21]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );



    ### Response
    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    `{question}`:
    ```sql3
    """


In [22]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT

In [23]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [24]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [34]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Samples
    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [35]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [27]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [28]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);


#Now the question in spanish.


In [36]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Devuelveme el nombre del empleado mejor pagado")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [37]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [38]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.